# Code to generate folder, modify 
May 8, 2023

In [1]:
import numpy as np
import os
import argparse
import subprocess as sp
import shutil
import glob

In [2]:
run_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_march14/dm_tests/runs/'
src='/g/g92/ayyar1/lsd_project_docs/su4_dm_grid_lsd/dm_tests/run_tioga/submit_2_dwhsdm.sh'


In [3]:

def f_build_submit_script(dict_pars,fname):
    
    submit_strg='''#!/bin/bash
################
# Begin LSF Directives
#SBATCH -A latticgc
#SBATCH -t 00:10:00
#SBATCH -J hsdm8nodes
#SBATCH -o hsdm8nodes.%J
#SBATCH -e hsdm8nodes.%J
#SBATCH -N {N}
#SBATCH -n 8
#SBATCH --exclusive
#SBATCH --gpu-bind=map_gpu:0,1,2,3,7,6,5,4
#SBATCH -c 8
###SBATCH --threads-per-core=1

echo "--start " `date` `date +%s`

export BIND="--cpu-bind=verbose,map_ldom:3,3,1,1,2,2,0,0"

export MPIR_CVAR_GPU_EAGER_DEVICE_MEM=16384
export MPICH_GPU_SUPPORT_ENABLED=1
export MPICH_SMP_SINGLE_COPY_MODE=CMA
export OMP_NUM_THREADS=8
export MPICH_OFI_NIC_POLICY=GPU
export OPT="--comms-concurrent --comms-overlap "

source $GRID_DIR/setup_env.sh
export TSAN_OPTIONS='ignore_noninstrumented_modules=1'
export LD_LIBRARY_PATH

Ls={ls}
traj_l={traj_l}
md_steps={md_steps}
BETA={beta}
M_F={mf}

APP="$RUN_DIR/dm_tests/build/dweofa_HSDM --grid 16.16.16.8 --mpi 4.4.2.2 --shm 2048 --shm-force-mpi 1 --device-mem 5000 --Trajectories 200 --Thermalizations 10 $OPT $Ls $traj_l $md_steps $BETA $M_F"
#srun --gpus-per-task 1 -n64 $BIND $APP > HSDM.out
#srun --gpus-per-task 1 -n64 $APP > HSDM.out
srun -n64 $APP > HSDM.out

echo "--end " `date` `date +%s`

    '''.format(**dict_pars)
    with open(fname,'w') as f: f.write(submit_strg)


In [9]:
Lx=16
Lt=8
beta=10.9
mf=0.6443


dict_pars={'ls':4, 'traj_l':1, 'md_steps':10, # ls=DWF 5th dim size
            'beta':beta, 'mf':mf , 
          'Lx':Lx, 'Lt':Lt, # Lattice size 
          'N':8  ## Number of nodes
          }


In [10]:
strg='run_Lx-%s_Lt-%s_beta-%s_mf-%s'%(Lx,Lt,beta,mf)
fldr=run_dir+'%s/'%(strg)

print(fldr)

if os.path.exists(fldr): 
    print("Error: Directory %s exists",fldr)
    
else: 
    os.makedirs(run_dir+'/{0}'.format(strg))
    shutil.copy(src,fldr)
    fname=fldr+'submit_2_dwhsdm.sh'
    f_build_submit_script(dict_pars,fname)
    
#     cmd='sed -i "34 s/BETA=.*/BETA=%s/" %s'%(beta,fname) 
#     print(cmd)
#     sp.check_output(cmd,shell=True)
#     cmd='sed -i "35 s/M_F=.*/M_F=%s/" %s'%(mf,fname) 
#     print(cmd)
#     sp.check_output(cmd,shell=True)
    
#     pass



/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_march14/dm_tests/runs/run_Lx-16_Lt-8_beta-10.9_mf-0.6443/


### Code to rename folders
os.chdir(run_dir)
fldr_lst=glob.glob('run_L*')
print(fldr_lst)
for src in fldr_lst:
     target=src.replace('run_Ls','run_Lx')
     print(src,target)
     os.rename(src,target)